We have 6 classes in this classification task:
    1. barren_land (73396)
    2. building (14922)
    3. grassland (50346)
    4. road (8191)
    5. trees (56808)
    6. water (120331)

As indicated above, they are hugely inbalanced

It's highly likely that a CNN with a classifier at the end will over predict water and under predict road
Since we don't know the natural distribution of these images, we will attempt to balance these images out

How:
    1. Augmentation
    2. Subsampling/oversampling
    3. ???
    

May not be the best practice will need to look up subsampling methods

In [1]:
count_by_cat = {'barren_land' :73396, 'building':14922, 'grassland':50346, 'road':8191, 'trees':56808, 'water':120331}

In [2]:
count_by_cat_perc = {k: v/sum(count_by_cat.values())*100 for k,v in count_by_cat.items()}

In [3]:
count_by_cat_perc

{'barren_land': 22.653505929122144,
 'building': 4.605640845200837,
 'grassland': 15.53917665141947,
 'road': 2.5281332370352536,
 'trees': 17.53365803070427,
 'water': 37.139885306518025}

In [4]:
sum(count_by_cat.values())

323994

Now let's try protyping some of this

In [5]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import models, transforms, utils
from PIL import Image
from tqdm import tqdm
from torch.optim import Adam
import math
import cv2

In [26]:
class vgg_preloaded(nn.Module):

	def __init__(self, num_class, use_cuda):
		super(vgg_preloaded, self).__init__()
		self.use_cuda = use_cuda
		self.num_class = num_class
		self.dtype = torch.cuda.FloatTensor if self.use_cuda else torch.FloatTensor
		model = models.inception_v3(pretrained=True)
		self.model = model.cuda() if self.use_cuda else model
		for param in self.model.features.parameters():
			param.require_grad = False
		num_features = self.model.classifier[6].in_features
		features = list(self.model.classifier.children())[:-1] # Remove last layer
		features.extend([nn.Linear(num_features, self.num_class)])
		self.model.classifier = nn.Sequential(*features)
		self.model.classifier.require_grad = True

	def forward(self, inp):
		return(self.model(inp))

In [2]:
image_dir = '../../data/images/'
barren_land_dir = image_dir + 'barren_land/'
barren_land = os.listdir(barren_land_dir)

In [7]:
image_3 = cv2.imread(barren_land_dir + barren_land[3])

In [8]:
image_3.shape

(28, 28, 3)

We need to check if everything is in png format

In [9]:
os.path.isdir(os.path.join(image_dir, os.listdir(image_dir)[0]))

False

In [10]:
# creating individual paths
sub_paths = [os.path.join(image_dir, file_dir) for file_dir in os.listdir(image_dir)]
# check if they're folders
sub_paths = [path for path in sub_paths if os.path.isdir(path)]

In [11]:
sub_paths

['../../data/images/water',
 '../../data/images/trees',
 '../../data/images/road',
 '../../data/images/barren_land',
 '../../data/images/building',
 '../../data/images/grassland']

In [12]:
from tqdm import tqdm
res = [0 if file_name[-3:] == 'png' else 1 for file_name in tqdm(os.listdir(sub_paths[0]))]

100%|██████████| 120332/120332 [00:00<00:00, 1425240.42it/s]


In [13]:
res = {}
for path in sub_paths:
    res[path] = sum([0 if file_name[-3:] == 'png' else 1 for file_name in tqdm(os.listdir(path))])

100%|██████████| 50347/50347 [00:00<00:00, 1394408.54it/s]


In [14]:
res

{'../../data/images/barren_land': 0,
 '../../data/images/building': 0,
 '../../data/images/grassland': 0,
 '../../data/images/road': 0,
 '../../data/images/trees': 0,
 '../../data/images/water': 0}

OK! 

so all our images are of '.png' format

In [15]:
sub_paths

['../../data/images/water',
 '../../data/images/trees',
 '../../data/images/road',
 '../../data/images/barren_land',
 '../../data/images/building',
 '../../data/images/grassland']

In [16]:
class_dict = {'water':0, 'trees':1, 'road':2, 'barren_land': 3, 'building': 4, 'grassland':5}

all_files_dict = {}
for path in sub_paths:
    print(os.path.split(path)[1])
    all_files_dict = {**all_files_dict, **{os.path.join(path, file_name):class_dict[os.path.split(path)[1]] for file_name in  os.listdir(path)}}

water
trees
road
barren_land
building
grassland


In [17]:
len(all_files_dict)

324000

Ok officially we have 32.4k images

In [18]:
set(all_files_dict.values())

{0, 1, 2, 3, 4, 5}

In [19]:
import pandas as pd

In [20]:
test_df = pd.DataFrame.from_dict({'col1': ['a','b','c','d'], 'col2':[1,2,3,4]})

In [21]:
test_df.loc[test_df['col1'] == 'a', 'col2']

0    1
Name: col2, dtype: int64

In [22]:
import numpy as np
nparrayres = np.array(test_df.loc[test_df['col1'] == 'a', 'col2'])
nparrayres

array([1])

In [23]:
torch.from_numpy(nparrayres)[0]

tensor(1)

Now we can prototype this on the .py file

In [24]:
list(all_files_dict.keys())[:15]

['../../data/images/water/59510.png',
 '../../data/images/water/27087.png',
 '../../data/images/water/21069.png',
 '../../data/images/water/20806.png',
 '../../data/images/water/65273.png',
 '../../data/images/water/67526.png',
 '../../data/images/water/110447.png',
 '../../data/images/water/118455.png',
 '../../data/images/water/63757.png',
 '../../data/images/water/73946.png',
 '../../data/images/water/111116.png',
 '../../data/images/water/14412.png',
 '../../data/images/water/19821.png',
 '../../data/images/water/24287.png',
 '../../data/images/water/28949.png']

In [25]:
sorted(list(all_files_dict.keys()))[:15]

['../../data/images/barren_land/0.png',
 '../../data/images/barren_land/1.png',
 '../../data/images/barren_land/10.png',
 '../../data/images/barren_land/100.png',
 '../../data/images/barren_land/1000.png',
 '../../data/images/barren_land/10000.png',
 '../../data/images/barren_land/10001.png',
 '../../data/images/barren_land/10002.png',
 '../../data/images/barren_land/10003.png',
 '../../data/images/barren_land/10004.png',
 '../../data/images/barren_land/10005.png',
 '../../data/images/barren_land/10006.png',
 '../../data/images/barren_land/10007.png',
 '../../data/images/barren_land/10008.png',
 '../../data/images/barren_land/10009.png']

In [26]:
torch.Tensor([0])[0]

tensor(0.)

In [27]:
torch.from_numpy(np.array(0))

tensor(0)

In [28]:
transforms.RandomVerticalFlip()

In [29]:
?transforms.RandomRotation()

In [29]:
vgg16 = models.vgg16(pretrained=True)

In [30]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(

In [6]:
inceptionv3 = models.inception_v3(pretrained=True)

In [7]:
inceptionv3

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [34]:
inceptionv3.fc.in_features

2048

In [41]:
for param in inceptionv3.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [36]:
inceptionv3.fc = nn.Linear(2048, 6)

In [37]:
for param in inceptionv3.parameters():
    param.requires_grad = False

In [38]:
inceptionv3.fc.requires_grad = True

In [39]:
for param in inceptionv3.fc.parameters():
    print(param)
    param.requires_grad = True
    print(param.requires_grad)

Parameter containing:
tensor([[ 4.9460e-03, -1.2930e-02,  1.1320e-02,  ..., -1.4274e-02,
         -5.8162e-03, -1.8230e-02],
        [-7.5780e-03, -9.3602e-03, -1.0560e-03,  ...,  2.1353e-02,
         -7.9112e-03,  1.1705e-02],
        [-5.3076e-03, -8.0373e-03,  1.5226e-02,  ..., -9.8826e-03,
          1.8469e-02,  7.3391e-03],
        [-7.1948e-04, -1.9954e-02, -4.8629e-03,  ...,  3.6628e-03,
         -7.1042e-03, -1.4692e-02],
        [ 1.4095e-02, -7.2652e-03,  1.9140e-02,  ..., -8.9945e-03,
         -1.8647e-02,  3.8504e-03],
        [ 3.3271e-03,  1.9414e-02,  3.3934e-04,  ..., -1.6170e-02,
         -1.0090e-02,  5.4354e-03]])
True
Parameter containing:
tensor([ 0.0123,  0.0077, -0.0182,  0.0163,  0.0101, -0.0191])
True


In [40]:
inceptionv3.train()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [42]:
from torch.optim import SGD

In [3]:
from custom_dset import custom_dset

In [4]:
dset = custom_dset(image_dir)

In [32]:
dset.all_files

['../../data/images/barren_land/0.png',
 '../../data/images/barren_land/1.png',
 '../../data/images/barren_land/10.png',
 '../../data/images/barren_land/100.png',
 '../../data/images/barren_land/1000.png',
 '../../data/images/barren_land/10000.png',
 '../../data/images/barren_land/10001.png',
 '../../data/images/barren_land/10002.png',
 '../../data/images/barren_land/10003.png',
 '../../data/images/barren_land/10004.png',
 '../../data/images/barren_land/10005.png',
 '../../data/images/barren_land/10006.png',
 '../../data/images/barren_land/10007.png',
 '../../data/images/barren_land/10008.png',
 '../../data/images/barren_land/10009.png',
 '../../data/images/barren_land/1001.png',
 '../../data/images/barren_land/10010.png',
 '../../data/images/barren_land/10011.png',
 '../../data/images/barren_land/10012.png',
 '../../data/images/barren_land/10013.png',
 '../../data/images/barren_land/10014.png',
 '../../data/images/barren_land/10015.png',
 '../../data/images/barren_land/10016.png',
 '.

In [75]:
class_sample_count = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0,}
for k,v in all_files_dict.items():
    class_sample_count[v] += 1

In [76]:
class_sample_count

{0: 120332, 1: 56809, 2: 8192, 3: 73397, 4: 14923, 5: 50347}

In [77]:
class_sample_count = [class_sample_count[key] for key in sorted(class_sample_count.keys())]

In [78]:
weights = 100000./torch.tensor(class_sample_count, dtype = torch.float)

In [79]:
weights

tensor([ 0.8310,  1.7603, 12.2070,  1.3625,  6.7011,  1.9862])

In [80]:
samples_weights = [weights[label] for label in dset.labels]

In [29]:
from torch.utils.data import WeightedRandomSampler

In [5]:
from collections import Counter

In [6]:
class_sample_count = Counter(dset.labels)

In [7]:
sorted_class_sample_count = [class_sample_count[key] for key in sorted(class_sample_count.keys())]

In [8]:
sorted_class_sample_count

[120332, 56809, 8192, 73397, 14923, 50347]

In [9]:
weights = 100000./torch.tensor(sorted_class_sample_count, dtype = torch.float)

In [10]:
samples_weights = [weights[label] for label in dset.labels]

In [11]:
samples_weights[:5]

[tensor(1.3625),
 tensor(1.3625),
 tensor(1.3625),
 tensor(1.3625),
 tensor(1.3625)]

In [12]:
from custom_dset import train_val_test_split
train_data, val_data, test_data = train_val_test_split(dset, train_split = 0.7, val_split = 0.2, test_split = 0.1)

In [15]:
train_class_sample_count = Counter(dset.dir_to_class_dict[dset.all_files[i]] for i in train_data.indices)

In [17]:
train_class_sample_count

Counter({0: 84055, 1: 39671, 2: 5741, 3: 51512, 4: 10513, 5: 35308})

In [19]:
sum(train_class_sample_count.values())/sum(sorted_class_sample_count)

0.7

In [20]:
sorted_train_class_sample_count = [train_class_sample_count[key] for key in sorted(train_class_sample_count.keys())]





In [21]:
sorted_train_class_sample_count

[84055, 39671, 5741, 51512, 10513, 35308]

In [23]:
weights = 100000./torch.tensor(sorted_train_class_sample_count, dtype = torch.float)

In [24]:
weights

tensor([ 1.1897,  2.5207, 17.4186,  1.9413,  9.5120,  2.8322])

In [25]:
samples_weights = [weights[label] for label in train_data.dataset.labels]

In [27]:
set(samples_weights)

{tensor(9.5120),
 tensor(1.1897),
 tensor(1.1897),
 tensor(1.1897),
 tensor(1.1897),
 tensor(2.5207),
 tensor(1.1897),
 tensor(1.1897),
 tensor(1.1897),
 tensor(1.1897),
 tensor(1.1897),
 tensor(1.9413),
 tensor(1.9413),
 tensor(1.1897),
 tensor(2.5207),
 tensor(1.1897),
 tensor(1.1897),
 tensor(2.5207),
 tensor(1.1897),
 tensor(2.5207),
 tensor(1.1897),
 tensor(1.1897),
 tensor(9.5120),
 tensor(1.1897),
 tensor(1.1897),
 tensor(2.5207),
 tensor(1.9413),
 tensor(1.1897),
 tensor(1.9413),
 tensor(1.9413),
 tensor(1.9413),
 tensor(1.9413),
 tensor(1.9413),
 tensor(9.5120),
 tensor(2.5207),
 tensor(2.8322),
 tensor(2.8322),
 tensor(17.4186),
 tensor(2.5207),
 tensor(1.9413),
 tensor(2.5207),
 tensor(1.1897),
 tensor(1.1897),
 tensor(2.5207),
 tensor(1.9413),
 tensor(1.1897),
 tensor(1.1897),
 tensor(2.5207),
 tensor(1.1897),
 tensor(2.5207),
 tensor(1.1897),
 tensor(1.1897),
 tensor(1.1897),
 tensor(1.1897),
 tensor(1.1897),
 tensor(2.5207),
 tensor(1.1897),
 tensor(1.1897),
 tensor(1.189

In [30]:
sampler = WeightedRandomSampler(weights=samples_weights,
                                num_samples=len(samples_weights),
                                replacement=True)

In [31]:
bs = 4
num_workers = 1
train_loader = torch.utils.data.DataLoader(train_data, batch_size=bs, sampler = sampler,
                                           num_workers=num_workers, pin_memory=False)

In [ ]:
train_data, val_data, test_data = train_val_test_split(dset, train_split = 0.7, val_split = 0.2, test_split = 0.1)